<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data" data-toc-modified-id="Data-1">Data</a></span><ul class="toc-item"><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-1.1">Load Data</a></span></li><li><span><a href="#Filter-Data" data-toc-modified-id="Filter-Data-1.2">Filter Data</a></span></li></ul></li><li><span><a href="#Readability" data-toc-modified-id="Readability-2">Readability</a></span></li><li><span><a href="#Tag-Articles" data-toc-modified-id="Tag-Articles-3">Tag Articles</a></span></li><li><span><a href="#NLP" data-toc-modified-id="NLP-4">NLP</a></span><ul class="toc-item"><li><span><a href="#Load-Roberta-Pipeline" data-toc-modified-id="Load-Roberta-Pipeline-4.1">Load Roberta Pipeline</a></span></li><li><span><a href="#Summary-Stats" data-toc-modified-id="Summary-Stats-4.2">Summary Stats</a></span></li><li><span><a href="#Yahoo" data-toc-modified-id="Yahoo-4.3">Yahoo</a></span></li><li><span><a href="#MarketWatch" data-toc-modified-id="MarketWatch-4.4">MarketWatch</a></span></li><li><span><a href="#PR-Newswire" data-toc-modified-id="PR-Newswire-4.5">PR Newswire</a></span></li></ul></li><li><span><a href="#Example" data-toc-modified-id="Example-5">Example</a></span></li></ul></div>

- git lfs install
- git clone https://huggingface.co/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis

In [1]:
# !git lfs install
# !git clone https://huggingface.co/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis
# !pip install transformers

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
import os
# cur_path = "/content/drive/MyDrive/Colab Notebooks/capstone/"
# os.chdir(cur_path)
# !pwd

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
from transformers import pipeline
import time
import datetime

## Data 

### Load Data

In [4]:
def load_data(): 
    path = r'/Users/TyPainter1/Desktop/Masters/spring-2022/capstone/00-data/gdelt_data/'
    #path = r'/content/drive/MyDrive/Colab Notebooks/capstone/gdelt_data/'
    years = os.listdir(path)
    years.remove('.DS_Store')
    years.remove('.ipynb_checkpoints')
    yr = [x for x in years if not (x.startswith('.'))]

    year_paths= []
    months=[]
    file_paths= []

    for i in range(len(years)): # enter year folder
        year_paths.append(path + years[i]) # year paths
        months.extend(os.listdir(year_paths[i])) # months of data in each year
    months = [x for x in months if not (x.startswith('.'))]
    for j in range(len(months)): # enter months in each year folder
        file_paths.append(year_paths[i] + "/" + months[j]) # file in year path
    
    return(file_paths)

### Filter Data

In [5]:
def filter_websites(file):
    df = pd.read_csv(file, index_col=[0]) # read in csv
    m_df = df[df.website=='marketwatch.com'] # filter for website
    p_df = df[df.website=='prnewswire.com'] # filter for website
    y_df = df[df.website=='yahoo.com'][0:10] # filter for website
    
    return(m_df, p_df, y_df)

## Readability

In [6]:
#  !pip install https://github.com/andreasvc/readability/tarball/master

In [7]:
import readability

In [8]:
def read_grades(article):
    if article == "NA":
        results = readability.getmeasures(article, lang='en')
        cols = list(results['readability grades'])
        vals = np.empty((1,9))
        vals[:] = np.nan
        scores = pd.DataFrame(data=vals,columns=cols)
    elif article == "":
        article = "NA"
        results = readability.getmeasures(article, lang='en')
        cols = list(results['readability grades'])
        vals = np.empty((1,9))
        vals[:] = np.nan
        scores = pd.DataFrame(data=vals,columns=cols)
    else: 
        results = readability.getmeasures(article, lang='en')
        cols = list(results['readability grades'])
        vals = list(results['readability grades'].values())
        vals =[[round(val,2)] for val in vals]
        vals = np.asarray(vals).T
        scores = pd.DataFrame(data=vals,columns=cols)
    return(scores)

In [9]:
def sentence_info(article):
    if article == "NA":
        results = readability.getmeasures(article, lang='en')
        cols = list(results['sentence info'])
        idx = [7,-2]
        cols = [cols[i] for i in idx]
        vals = np.empty((1,2))
        vals[:] = np.nan
        scores = pd.DataFrame(data=vals,columns=cols)
    elif article == "":
        article = "NA"
        results = readability.getmeasures(article, lang='en')
        cols = list(results['sentence info'])
        idx = [7,-2]
        cols = [cols[i] for i in idx]
        vals = np.empty((1,2))
        vals[:] = np.nan
        scores = pd.DataFrame(data=vals,columns=cols)
    else:
        results = readability.getmeasures(article, lang='en')
        cols = list(results['sentence info'])
        idx = [7,-2]
        cols = [cols[i] for i in idx]
        vals = list(results['sentence info'].values())
        vals = [[vals[i]] for i in idx]
        vals = np.asarray(vals).T
        scores = pd.DataFrame(data=vals,columns=cols)
    return(scores)

## Tag Articles

In [10]:
# !pip install fuzzywuzzy[speedup]

In [11]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [12]:
tick_comp_df = pd.read_csv("../00-data/company_tickers.csv")
tick_comp_df.head()

,Symbol,Security
0,MMM,3M
1,AOS,A. O. Smith
2,ABT,Abbott
3,ABBV,AbbVie
4,ABMD,Abiomed


In [13]:
companies = tick_comp_df["Security"].to_list()
companies

['3M',
 'A. O. Smith',
 'Abbott',
 'AbbVie',
 'Abiomed',
 'Accenture',
 'Activision Blizzard',
 'ADM',
 'Adobe',
 'ADP',
 'Advance Auto Parts',
 'AES',
 'Aflac',
 'Agilent Technologies',
 'AIG',
 'Air Products',
 'Akamai',
 'Alaska Air Group',
 'Albemarle',
 'Alexandria',
 'Align',
 'Allegion',
 'Alliant Energy',
 'Allstate',
 'Alphabet (Class A)',
 'Alphabet (Class C)',
 'Altria',
 'Amazon',
 'Amcor',
 'AMD',
 'Ameren',
 'American Airlines Group',
 'American Electric Power',
 'American Express',
 'American Tower',
 'American Water',
 'Ameriprise Financial',
 'AmerisourceBergen',
 'Ametek',
 'Amgen',
 'Amphenol',
 'Analog Devices',
 'Ansys',
 'Anthem',
 'Aon',
 'APA Corporation',
 'Apple',
 'Applied Materials',
 'Aptiv',
 'Arista',
 'Assurant',
 'AT&T',
 'Atmos Energy',
 'Autodesk',
 'AutoZone',
 'AvalonBay Communities',
 'Avery Dennison',
 'Baker Hughes',
 'Ball',
 'Bank of America',
 'Bath & Body Works',
 'Baxter',
 'Becton Dickinson',
 'Berkley',
 'Berkshire Hathaway',
 'Best Buy',


In [14]:
def tag_article(article, companies, df): #, tickers
    if (article == "NA") | (article==""):
        tags = np.nan
    else:
        scores = process.extract(article, 
                                 companies,
                                 scorer=fuzz.token_set_ratio, 
                                 limit=50)
        comp_tags = [scores[i][0] for i in range(len(scores)) if scores[i][1]>90]
        tags = list(df.loc[df.Security.isin(comp_tags), "Symbol"])
        if len(tags) == 0:
            tags = np.nan
    return(tags)

In [15]:
def row_tags(old_df):
    old_df = old_df.reset_index(drop=True)
    new_df = pd.DataFrame(columns=old_df.columns)
    for i in range(len(old_df)):
        new_tags = old_df.tags[i]#.strip("][").replace("'","").split(', ')
        for t in new_tags:
            new_df = new_df.append(old_df.iloc[i,:-1]).reset_index(drop=True)
            new_df.tags.iloc[-1] = t
    return(new_df)

## NLP

### Load Roberta Pipeline

In [16]:
roberta = pipeline(task='sentiment-analysis', 
                   model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis",
                   return_all_scores=True,
                   max_length=514, 
                   truncation=True)

 ### Summary Stats

In [17]:
def summary_stats(score):
    if len(score)== 0:
        avg = np.nan
        med = np.nan
        min = np.nan
        max = np.nan
    else:
        avg = round(np.mean(score),2)
        med = round(np.median(score),2)
        min = round(np.min(score),2)
        max = round(np.max(score),2)
    return(avg, med, min, max)

### Yahoo

In [18]:
# yahoo web scrape NLP
def yahoo_nlp(urls, df):

    df["pos_mean"] = np.nan
    df["pos_median"] = np.nan
    df["pos_min"] = np.nan
    df["pos_max"] = np.nan

    df["neg_mean"] = np.nan
    df["neg_median"] = np.nan
    df["neg_min"] = np.nan
    df["neg_max"] = np.nan

    df["neu_mean"] = np.nan
    df["neu_median"] = np.nan
    df["neu_min"] = np.nan
    df["neu_max"] = np.nan
    
    df["net_mean"] = np.nan
    df["net_median"] = np.nan
    df["net_min"] = np.nan
    df["net_max"] = np.nan
    
    score_df = pd.DataFrame()
    sentence_df = pd.DataFrame()
    tag_df = pd.DataFrame(columns=["tags"])
    
    nf = "Content is currently unavailable"
    
    for i in range(len(urls)): #filter_urls
        URL = urls.iloc[i] # filter_urls
        page = requests.get(URL,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})
        soup = BeautifulSoup(page.content, "html.parser")
        if nf in soup.prettify():
            text = "NA"
            score_df = score_df.append(read_grades(text))
            sentence_df = sentence_df.append(sentence_info(text))
            tag_df.at[i, 'tags'] = tag_article(text,companies,tick_comp_df)
        
            continue
        try:
            try:
                master = soup.find(id="Masterwrap")
                final = master.find("div", class_="caas-body")
            except AttributeError:
                master = soup.find(id="Masterwrap2Col")
                final = master.find("div", class_="caas-body")
        except AttributeError:
            pass
        
        text = re.sub('(<p>)', ' ', str(final))
        text = re.sub('<[^>]+>', '', str(text))
        sen_list = text.split(". ")[:-2]
        article = [".\n ".join(sen_list[:-2])][0] # remove author signature
        score_df = score_df.append(read_grades(article))
        sentence_df = sentence_df.append(sentence_info(article))
        tag_df.at[i, 'tags'] = tag_article(article,companies,tick_comp_df)

        pos = []
        neg = []
        neu = []
        net = []

        for sen in sen_list:
            try:
                output = roberta(sen)
            except IndexError:
                output = roberta(sen[0:514])
            neg.append(output[0][0]['score'])
            neu.append(output[0][1]['score'])
            pos.append(output[0][2]['score'])
            net.append(pos[-1] - neg[-1])
        
        df.pos_mean.iloc[i], df.pos_median.iloc[i], df.pos_min.iloc[i], df.pos_max.iloc[i] = summary_stats(pos)
        df.neg_mean.iloc[i], df.neg_median.iloc[i], df.neg_min.iloc[i], df.neg_max.iloc[i] = summary_stats(neg)
        df.neu_mean.iloc[i], df.neu_median.iloc[i], df.neu_min.iloc[i], df.neu_max.iloc[i] = summary_stats(neu)
        df.net_mean.iloc[i], df.net_median.iloc[i], df.net_min.iloc[i], df.net_max.iloc[i] = summary_stats(net)
    
    score_df.index = df.index
    sentence_df.index = df.index
    tag_df.index = df.index
    df = pd.concat([df,score_df,sentence_df,tag_df],axis=1)
    df = df.dropna()
        
    return(df)

### MarketWatch

In [19]:
# marketwatch web scrape NLP
def mwatch_nlp(urls, df):
    
    df["pos_mean"] = np.nan
    df["pos_median"] = np.nan
    df["pos_min"] = np.nan
    df["pos_max"] = np.nan

    df["neg_mean"] = np.nan
    df["neg_median"] = np.nan
    df["neg_min"] = np.nan
    df["neg_max"] = np.nan

    df["neu_mean"] = np.nan
    df["neu_median"] = np.nan
    df["neu_min"] = np.nan
    df["neu_max"] = np.nan
    
    df["net_mean"] = np.nan
    df["net_median"] = np.nan
    df["net_min"] = np.nan
    df["net_max"] = np.nan
    
    score_df = pd.DataFrame()
    sentence_df = pd.DataFrame()
    tag_df = pd.DataFrame(columns=["tags"])
    
    nf = "Story not found"
    
    for i in range(len(urls)): #filter_urls
        URL = urls.iloc[i] # filter_urls
        page = requests.get(URL,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})
        soup = BeautifulSoup(page.content, "html.parser")
        if nf in soup.prettify():
            text = "NA"
            score_df = score_df.append(read_grades(text))
            sentence_df = sentence_df.append(sentence_info(text))
            tag_df.at[i, 'tags'] = tag_article(text,companies,tick_comp_df)
            continue
        try:
            master = soup.find(id="maincontent")
            final = master.find("div", class_="article__body article-wrap at16-col16 barrons-article-wrap")
            text = final.text.strip()
        except AttributeError:
            pass
        sen_list = text.split(".")
        article = [".\n ".join(sen_list)][0]
        score_df = score_df.append(read_grades(article))
        sentence_df = sentence_df.append(sentence_info(article))
        tag_df.at[i, 'tags'] = tag_article(article,companies,tick_comp_df)

        pos = []
        neg = []
        neu = []
        net = []

        for sen in sen_list:
            try:
                output = roberta(sen)
            except IndexError:
                output = roberta(sen[0:514])
            neg.append(output[0][0]['score'])
            neu.append(output[0][1]['score'])
            pos.append(output[0][2]['score'])
            net.append(pos[-1] - neg[-1])

        df.pos_mean.iloc[i], df.pos_median.iloc[i], df.pos_min.iloc[i], df.pos_max.iloc[i] = summary_stats(pos)
        df.neg_mean.iloc[i], df.neg_median.iloc[i], df.neg_min.iloc[i], df.neg_max.iloc[i] = summary_stats(neg)
        df.neu_mean.iloc[i], df.neu_median.iloc[i], df.neu_min.iloc[i], df.neu_max.iloc[i] = summary_stats(neu)
        df.net_mean.iloc[i], df.net_median.iloc[i], df.net_min.iloc[i], df.net_max.iloc[i] = summary_stats(net)
    
    score_df.index = df.index
    sentence_df.index = df.index
    tag_df.index = df.index
    df = pd.concat([df,score_df,sentence_df,tag_df],axis=1)
    df = df.dropna()
        
    return(df)

### PR Newswire

In [20]:
# prnewswire web scrape NLP
def prnw_nlp(urls, df):
    
    df["pos_mean"] = np.nan
    df["pos_median"] = np.nan
    df["pos_min"] = np.nan
    df["pos_max"] = np.nan

    df["neg_mean"] = np.nan
    df["neg_median"] = np.nan
    df["neg_min"] = np.nan
    df["neg_max"] = np.nan

    df["neu_mean"] = np.nan
    df["neu_median"] = np.nan
    df["neu_min"] = np.nan
    df["neu_max"] = np.nan
    
    df["net_mean"] = np.nan
    df["net_median"] = np.nan
    df["net_min"] = np.nan
    df["net_max"] = np.nan
    
    sentence_df = pd.DataFrame()
    score_df = pd.DataFrame()
    tag_df = pd.DataFrame(columns=["tags"])
    
    nf = "Content is currently unavailable"
    
    for i in range(len(urls)): #filter_urls
        URL = urls.iloc[i] # filter_urls
        page = requests.get(URL,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})
        soup = BeautifulSoup(page.content, "html.parser")
        if nf in soup.prettify():
            text = "NA"
            score_df = score_df.append(read_grades(text))
            sentence_df = sentence_df.append(sentence_info(text))
            tag_df.at[i, 'tags'] = tag_article(text,companies,tick_comp_df)
            continue
        try:
            master = soup.find(id="main")
            final = master.find_all("div", class_="col-sm-10 col-sm-offset-1")
        except AttributeError:
            pass
        try:
            paragraphs = []
            for wrapper in final:
                paragraphs.append(wrapper.text.strip())
                text = [" ".join(paragraphs)][0]
        except:
            pass
        sen_list = text.split(". ")
        article = [".\n ".join(sen_list)][0]
        score_df = score_df.append(read_grades(article))
        sentence_df = sentence_df.append(sentence_info(article))
        tag_df.at[i, 'tags'] = tag_article(article,companies,tick_comp_df)

        pos = []
        neg = []
        neu = []
        net = []

        for sen in sen_list:
            try:
                output = roberta(sen)
            except IndexError:
                output = roberta(sen[0:514])
            neg.append(output[0][0]['score'])
            neu.append(output[0][1]['score'])
            pos.append(output[0][2]['score'])
            net.append(pos[-1] - neg[-1])
        df.pos_mean.iloc[i], df.pos_median.iloc[i], df.pos_min.iloc[i], df.pos_max.iloc[i] = summary_stats(pos)
        df.neg_mean.iloc[i], df.neg_median.iloc[i], df.neg_min.iloc[i], df.neg_max.iloc[i] = summary_stats(neg)
        df.neu_mean.iloc[i], df.neu_median.iloc[i], df.neu_min.iloc[i], df.neu_max.iloc[i] = summary_stats(neu)
        df.net_mean.iloc[i], df.net_median.iloc[i], df.net_min.iloc[i], df.net_max.iloc[i] = summary_stats(net)
    
    score_df.index = df.index
    sentence_df.index = df.index
    tag_df.index = df.index
    df = pd.concat([df,score_df,sentence_df,tag_df],axis=1)
    df = df.dropna()
        
    return(df)

## Example

In [21]:
# append yahoo, marketwatch, and prnewswire together for entire month dataset
def append_df(df1,df2,df3):
    df = df1.append([df2,df3])
    return(df)

In [28]:
file_paths[0]

'/Users/TyPainter1/Desktop/Masters/spring-2022/capstone/00-data/gdelt_data/2021/gdelt_apr2021.csv'

In [35]:
tic = time.perf_counter()
file_paths = load_data()
print("file_paths loaded")
for path in file_paths[0:1]:
    print(path)
    m_df, p_df, y_df = filter_websites(path)
    print("filter websites")
    m_df = m_df[0:10]
    p_df = p_df[0:5]
    y_df = y_df[0:5]
    
    m_url = m_df.url
    p_url = p_df.url
    y_url = y_df.url
    print("URL")
    
    mwatch_df = mwatch_nlp(m_url, m_df)
    print("mwatch NLP")
    prnw_df = prnw_nlp(p_url, p_df)
    print("prnw NLP")
    yahoo_df = yahoo_nlp(y_url, y_df)
    print("yahoo NLP")
    
    app_df = append_df(yahoo_df,mwatch_df, prnw_df)
    print("Append")
    
    month_df = row_tags(app_df)
    print("Row tags")
    
    year_number = str(month_df.year.unique().item())
    month_number = str(month_df.month.unique().item())
    datetime_object = datetime.datetime.strptime(month_number, "%m")
    month_name = datetime_object.strftime("%b").lower()
    file_name = 'nlp_'+month_name+str(year_number)+'.csv'
    month_df.to_csv(file_name,
                   index=False)
    
toc = time.perf_counter()
print(f"Completed in {(toc - tic)/60:0.2f} minutes")

file_paths loaded
/Users/TyPainter1/Desktop/Masters/spring-2022/capstone/00-data/gdelt_data/2021/gdelt_apr2021.csv
filter websites
URL
mwatch NLP
prnw NLP
yahoo NLP
Append
Row tags
Completed in 0.95 minutes


In [23]:
! pip install line_profiler
%load_ext line_profiler

     |████████████████████████████████| 53 kB 1.8 MB/s eta 0:00:011


In [36]:
m_url

50    https://www.marketwatch.com/story/biden-seeks-...
51    https://www.marketwatch.com/story/marathon-oil...
59    https://www.marketwatch.com/story/paul-simon-s...
61    https://www.marketwatch.com/story/netflix-fork...
66    https://www.marketwatch.com/story/south-koreas...
84    https://www.marketwatch.com/story/chauvin-told...
88    https://www.marketwatch.com/story/pentagon-rel...
94    https://www.marketwatch.com/story/u-s-alarmed-...
96    https://www.marketwatch.com/story/sarah-palin-...
99    https://www.marketwatch.com/story/myanmar-coul...
Name: url, dtype: object

In [37]:
m_df

,year,month,day,website,url,pos_mean,pos_median,pos_min,pos_max,neg_mean,...,neg_min,neg_max,neu_mean,neu_median,neu_min,neu_max,net_mean,net_median,net_min,net_max
50,2021,4,1,marketwatch.com,https://www.marketwatch.com/story/biden-seeks-...,0.24,0.0,0.0,1.00,0.00,...,0.0,0.00,0.76,1.0,0.00,1.0,0.24,-0.0,-0.00,1.00
51,2021,4,1,marketwatch.com,https://www.marketwatch.com/story/marathon-oil...,0.00,0.0,0.0,0.00,0.09,...,0.0,1.00,0.91,1.0,0.00,1.0,-0.09,-0.0,-1.00,0.00
59,2021,4,1,marketwatch.com,https://www.marketwatch.com/story/paul-simon-s...,0.17,0.0,0.0,1.00,0.00,...,0.0,0.00,0.83,1.0,0.00,1.0,0.17,-0.0,-0.00,1.00
61,2021,4,1,marketwatch.com,https://www.marketwatch.com/story/netflix-fork...,0.22,0.0,0.0,1.00,0.00,...,0.0,0.00,0.78,1.0,0.00,1.0,0.22,0.0,-0.00,1.00
66,2021,4,1,marketwatch.com,https://www.marketwatch.com/story/south-koreas...,0.57,1.0,0.0,1.00,0.01,...,0.0,0.16,0.42,0.0,0.00,1.0,0.56,1.0,-0.16,1.00
84,2021,4,1,marketwatch.com,https://www.marketwatch.com/story/chauvin-told...,0.00,0.0,0.0,0.02,0.07,...,0.0,0.69,0.93,1.0,0.31,1.0,-0.06,-0.0,-0.69,0.00
88,2021,4,1,marketwatch.com,https://www.marketwatch.com/story/pentagon-rel...,0.17,0.0,0.0,1.00,0.10,...,0.0,1.00,0.73,1.0,0.00,1.0,0.07,-0.0,-1.00,1.00
94,2021,4,1,marketwatch.com,https://www.marketwatch.com/story/u-s-alarmed-...,0.04,0.0,0.0,0.90,0.20,...,0.0,0.99,0.76,1.0,0.00,1.0,-0.16,-0.0,-0.98,0.89
96,2021,4,1,marketwatch.com,https://www.marketwatch.com/story/sarah-palin-...,0.08,0.0,0.0,0.85,0.00,...,0.0,0.00,0.92,1.0,0.15,1.0,0.08,-0.0,-0.00,0.84
99,2021,4,1,marketwatch.com,https://www.marketwatch.com/story/myanmar-coul...,0.05,0.0,0.0,1.00,0.24,...,0.0,1.00,0.71,1.0,0.00,1.0,-0.19,-0.0,-1.00,1.00


In [38]:
%lprun -f mwatch_nlp mwatch_nlp(m_url, m_df)

/Users/TyPainter1/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
